In [144]:
# Import required libraries
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import automl
from azure.ai.ml import Input

import json
import pandas as pd

In [148]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential


credential = InteractiveBrowserCredential()

In [150]:
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "98dcfdf0-c14b-4a76-9961-31fff2d85560"
    resource_group = "sales-forecast"
    workspace = "sales-forecast-ws1"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

Found the config file in: .\config.json


In [152]:
workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

In [153]:
workspace = ml_client.workspaces.get(name=ml_client.workspace_name)

output = {}
output["Workspace"] = ml_client.workspace_name
output["Subscription ID"] = ml_client.connections._subscription_id
output["Resource Group"] = workspace.resource_group
output["Location"] = workspace.location

{'Workspace': 'sales-forecast-ws1',
 'Subscription ID': '98dcfdf0-c14b-4a76-9961-31fff2d85560',
 'Resource Group': 'sales-forecast',
 'Location': 'eastus2'}

In [154]:
pd.set_option("display.max_colwidth", None)
outputDf = pd.DataFrame(data=output, index=[""])
outputDf.T

,
Workspace,sales-forecast-ws1
Subscription ID,98dcfdf0-c14b-4a76-9961-31fff2d85560
Resource Group,sales-forecast
Location,eastus2


In [165]:
import shutil
import os
import yaml


def create_ml_table(csv_file, output, delimiter=",", encoding="ascii"):
    os.makedirs(output, exist_ok=True)
    fname = os.path.split(csv_file)[-1]
    mltable = {
        "paths": [{"file": f"./{fname}"}],
        "transformations": [
            {"read_delimited": {"delimiter": delimiter, "encoding": encoding}}
        ],
    }
    with open(os.path.join(output, "MLTable"), "w") as f:
        f.write(yaml.dump(mltable))
    shutil.copy(csv_file, os.path.join(output, fname))

In [166]:
create_ml_table("train.csv", "./data/training-mltable-folder")

In [168]:
# Training MLTable defined locally, with local data to be uploaded
my_training_data_input = Input(
    type=AssetTypes.MLTABLE, path="./data/training-mltable-folder"
)

In [169]:
from azure.core.exceptions import ResourceNotFoundError
from azure.ai.ml.entities import AmlCompute

cluster_name = "sales-forecast-compute"

try:
    # Retrieve an already attached Azure Machine Learning Compute.
    compute = ml_client.compute.get(cluster_name)
except ResourceNotFoundError as e:
    compute = AmlCompute(
        name=cluster_name,
        size="STANDARD_DS3_V2",
        type="amlcompute",
        min_instances=0,
        max_instances=4,
        idle_time_before_scale_down=120,
    )
    poller = ml_client.begin_create_or_update(compute)
    poller.wait()

In [170]:
exp_name="sales_forecast_exp1"

forecasting_job = automl.forecasting(
    compute=cluster_name,
    experiment_name=exp_name,
    training_data=my_training_data_input,
    target_column_name="sales",
    primary_metric="NormalizedRootMeanSquaredError",
    n_cross_validations="auto",
    enable_model_explainability=True,
)

In [171]:
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(forecasting_job)  # submit the job to the backend

print(f"Created job: {returned_job}")

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy 'C:\Users\Dinesh.Sreekanthan\PycharmProjects\azure-sales-forecast\data\training-mltable-folder' 'https://salesforecastw1977535163.blob.core.windows.net/azureml-blobstore-1b1d8e1e-3008-4fb5-b35b-eee7668847c2/LocalUpload/47e045c56ae46181ddb7106d4d7bf204/training-mltable-folder' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading training-mltable-folder (121.8 MBs): 100%|################| 121800480/121800480 [00:40<00:00, 3000906.80it/s]




AttributeError: 'NoneType' object has no attribute '_to_rest_object'